# Could we have got there quicker?
## The power of laziness

We are now onto the 3rd look at this horrible dataset and whilst we have a better answer than just gun hoe and we have some better support for some of our assumptions and eye ball test bullshit, did we still mow straight over the obvious and justify it as using the eye ball test and some more fancy transformation of the data?

If you're like me, I may have bailed out before I got this far with this discussion, however, for anyone still waiting for the punch line and having the mental endurance to last this long, your reward is absa-freakin luttly we could have got a solid answer, compeltely justifed with much less work and I would say, an even more warm and fuzzy feeling solution as well as better insight and explainability.

Shit! that sounds like gold flakes will start raining down or something, but the point is, there is atleast one tool I use to cut through the bullshit quickly and in the spirit of back of the envelope calculations, provides expediance and surprisingly accuracte and easy to explain results.

Ok, enough woffling, here we go.....

In [1]:
# Some imports to get us active
from utils.tools import *

In [2]:
dfInbound_tonnes = pd.read_pickle('data/inbound_tonnes2010010120201101.pkl')
print('Data shape: {}'.format(dfInbound_tonnes.shape))
dfInbound_tonnes.head()


Data shape: (1767, 5)


,total_actualtonnes,plannedreceivals,totalactualcycles,Totalplannedcycles,delta
date,,,,,
2016-01-01,501223,542116,60,64,0.0
2016-01-02,414366,471262,50,56,-86857.0
2016-01-03,356261,401661,44,49,-58105.0
2016-01-04,459762,463961,56,56,103501.0
2016-01-05,344504,462463,43,57,-115258.0


The short and sweet answer is, we only need to ask one question before we start anything with time series data, what type of evolution over time does the data series have? Now I'm not taking a crap on your expectation and the lead up being deflated by this seemingly dull and slightly obtuse question, but this is it, does the data typically move upwards/downwards, flip back and forth or is on a [random walk](https://en.wikipedia.org/wiki/Random_walk). Knowing with information will allow us to quickly discount alot of analysis and modelling candidates and provide a guide on the fastest (and lazyest way to get the job done).

One measure to rule them all ![one measure](img/one-ring.jpg)

[The Hurst exponent](https://en.wikipedia.org/wiki/Hurst_exponent), in a nutshell, we calculate the Hurst exponent for a time series and compare it to the ranges below. This provides our slice through the details to know what we are dealing with. 
> $H = 0.5$ — Brownian motion <br>
> $0.5 < H < 1.0$ — persistent behavior <br>
> $0 < H < 0.5$ — anti-persistent behavior


I can hear the screamers now, oh but it doesn't do this and there is this analysis and you're full of crap and my mum said you're a troglodyte and so on. Feel free to have a nap to recover from this trauma then, but you will find if you use this method upfront, you can get back to your crisps or fags or vodka quicker, than if you don't.

In [110]:
# Let's run the data set through the function and see what we get

H,c,data = compute_Hc(dfInbound_tonnes.total_actualtonnes, kind='change', simplified=False)
print('Hurst exponent ch: {0}, Coeff: {1}\n'.format(H,c))

H,c,data = compute_Hc(dfInbound_tonnes.total_actualtonnes-dfInbound_tonnes.total_actualtonnes.mean(), kind='change', simplified=False)
print('Hurst exponent wn->rw: {0}, Coeff: {1}\n'.format(H,c))

H,c,data = compute_Hc(dfInbound_tonnes.total_actualtonnes, kind='random_walk', simplified=False)
print('Hurst exponent rw: {0}, Coeff: {1}\n'.format(H,c))

H,c,data = compute_Hc(dfInbound_tonnes.total_actualtonnes.cumsum(), kind='price', simplified=False)
print('Hurst exponent pr: {0}, Coeff: {1}\n'.format(H,c))


Hurst exponent ch: 0.5622493572029746, Coeff: 1.1156813449545677

Hurst exponent wn->rw: 0.5622493572029746, Coeff: 1.1156813449545677

Hurst exponent rw: 0.1919741808613579, Coeff: 2.1795255871803927

Hurst exponent pr: 0.8067781712826502, Coeff: 0.5040377725099195



A quick explination of the 3 different values and the kind parameter....blah blah blah


Comparing this with the range, we see for change, there is some persistant behaviour, not much tough. For the comparison to random walk, we see little association and a low value less than 0.5 indicates the data tends to revert to the mean rather than trending. The price version indicates we do see clear trending in the cummulative summation of the data, which correlates with the conclusion from the change value. After a while you'll get a feel for how much information these values contain and how much direct insight into the nature of the time series can be inferred from this one metric, albeit with 3 favours.

The conclusion is that the data tends to revert to the mean most of the time, with some trend that appears to be in a consistant direction. From this, we can immediately rule out models that include cyclic patterns and strongly trending approaches, as there is trend present, just not very much.